In [ ]:
import os
import torch
import torch.utils.data
from torch import nn, optim
from torch.autograd import Variable
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image

from VAE import VAE
from trainer import train, test

In [ ]:
# configuration
CUDA = True
SEED = 1
BATCH_SIZE = 128
LOG_INTERVAL = 10
EPOCHS = 10
# connections through the autoencoder bottleneck
ZDIMS = 10

In [ ]:
torch.manual_seed(SEED)
if CUDA:
    torch.cuda.manual_seed(SEED)
    
# DataLoader instances will load tensors directly into GPU memory
kwargs = {'num_workers': 1, 'pin_memory':True} if CUDA else {}

In [ ]:
# Download or load downloaded MNIST dataset
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('data', train=True, download=True, transform=transforms.ToTensor()),
    batch_size=BATCH_SIZE
    shuffle=True,
    **kwargs)

# Same for test data
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('data', train=False, transform=transforms.ToTensor()),
    batch_size=BATCH_SIZE
    shuffle=True,
    **kwargs)

In [ ]:
model = VAE()
if CUDA:
    model.cuda()

optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [ ]:
for epoch in range(1, Epochs + 1):
    train(epoch)
    test(epoch)
    
    # 64 sets of random ZDIMS-float vectors, i.e. 64 locations / MNIST digits in latent space
    sample = Variable(torch.randn(64, ZDIMS))
    if CUDA:
        sample = sample.cuda()
    sample = model.decode(sample).cpu()
    
    # save out as an 8x8 matrix of MNIST digits
    save_image(sample.data.viev(64, 1, 28, 28), 'results/sample ' + str(epoch) + '.png')